In [ ]:
!pip install openai wikipedia-api tiktoken transformers

In [ ]:
# OPENAI KEY lesen
import os
try:
    from google.colab import userdata
    OPENAI_KEY = userdata.get('OPENAI_KEY')
except:
    OPENAI_KEY = os.getenv('OPENAI_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_KEY


# Beispiel-Dokumente von Wikipedia

In [ ]:
import tiktoken
import wikipediaapi
from pathlib import Path
import time
from datetime import datetime

In [ ]:
page_name = 'Elvis_Presley'

wiki = wikipediaapi.Wikipedia('LangChain RAG', 'de', extract_format=wikipediaapi.ExtractFormat.WIKI)
text = wiki.page(page_name).text

In [ ]:
len(text)

In [ ]:
from openai import OpenAI
client = OpenAI()

## Übersetzen

Als Beispiel wollen wir den `Elvis` Artikel übersetzen. Herausforderung dabei? Der Artikel ist sehr lang und wir bekommen Probleme mit der Kontext-Länge von `GPT-3.5 Turbo`. Es erscheint eine Fehlermeldung.

In [ ]:
prompt = f'''{text}
Übersetze den folgenden Text ins englische:'''

In [ ]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
    )

completion.choices[0].message.content

### Tokens zählen

siehe auch den Tokenizer auf der OpenAI Website: https://platform.openai.com/tokenizer

Um das Problem zu umgehen kürzen wir den Artikel - dazu zählen wir mit `tiktoken` die Anzahl der Tokens des Textes und schneiden ihn nach 15.000 Tokens ab.

In [ ]:
encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')
len(encoding.encode(text))

In [ ]:
# wir schneiden den text nach n-Tokens ab
n = 15000
tokenized_text = encoding.encode(text)
tokenized_text = tokenized_text[:n]

tokenized_text[:10], text[:100]

In [ ]:
encoding.decode([6719])

Damit wir den gekürzten Text (sind jetzt ja Tokens anstatt lesbarer Text) wieder in den Prompt "stecken" können, wandeln wir ihn wieder in Text um. Gekürzt sollte jetzt auch keine Fehlermeldung mehr erscheinen!

In [ ]:
text = encoding.decode(tokenized_text)
text

In [ ]:
prompt = f'''{text}

Übersetze den Text vollständig ins englische:'''

In [ ]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}],
    max_tokens=None
    )

completion.choices[0].message.content

In [ ]:
len(encoding.encode(completion.choices[0].message.content))

#### "finish_reason"

In der Ausgabe der OpenAI API bekommen wir das Attribut "finish_reason" zurück. Das gibt an wieso die Ausgabe beendet wurde. 
- `stop` bedeutet, dass das Modell fertig ist und nicht mehr Text erzeugen möchte
- `length` bedeutet, dass die Kontext-Länge beim erzeugen des Textes überschritten wurde und der Text abgeschnitten ist

Hier sollte "length" erscheinen - das passiert aber nicht immer 

In [ ]:
completion.choices[0].finish_reason

## Geschwindigkeit

In [ ]:
prompt = 'Kennst du ein Donauwellenrezept?'

### gpt-3.5-turbo

In [ ]:
start = datetime.now()
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
    )
duration = datetime.now() - start

# Dauer in sekunden , Anzahl der generierten Tokens
duration.total_seconds(), completion.usage.completion_tokens

In [ ]:
# tokens per second
completion.usage.completion_tokens / duration.total_seconds()

### gpt-4-turbo

In [ ]:
start = datetime.now()
completion = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[{"role": "user", "content": prompt}]
    )
duration = datetime.now() - start

# Dauer in sekunden , Anzahl der generierten Tokens
duration.total_seconds(), completion.usage.completion_tokens

In [ ]:
# tokens per second
completion.usage.completion_tokens / duration.total_seconds()

### Streaming

In [ ]:
start_time = time.time()
response = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[{"role": "user", "content": prompt}],
    stream=True    
    )

# create variables to collect the stream of chunks
collected_chunks = []
collected_messages = []
# iterate through the stream of events
for chunk in response:
    chunk_time = time.time() - start_time  # calculate the time delay of the chunk
    collected_chunks.append(chunk)  # save the event response
    chunk_message = chunk.choices[0].delta.content  # extract the message
    collected_messages.append(chunk_message)  # save the message
    print(f"Message received {chunk_time:.2f} seconds after request: {chunk_message}")  # print the delay and text

# print the time delay and text received
print(f"Full response received {chunk_time:.2f} seconds after request")
# clean None in collected_messages
collected_messages = [m for m in collected_messages if m is not None]
full_reply_content = ''.join([m for m in collected_messages])
print(f"Full conversation received: {full_reply_content}")

## Model input / output (Huggingface / OpenSource Modelle)

![llm.png](images/llm.png)

### Wie kommen wir an den "neuen" Token?

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# Huggingface Modell laden
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype=torch.float32, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

In [ ]:
# Prompt definieren + tokenizen
inputs = tokenizer('''Elvis Aaron Presley (* January 8, 1935 in''', return_tensors="pt", return_attention_mask=False)
inputs['input_ids'].shape

In [ ]:
# wir geben die inputs in das Modell - die outputs verwenden wir um den neuen Token zu erhalten
outputs = model(**inputs)
outputs.logits.shape

In [ ]:
# wir suchen den letzten Output des Modells und bekommen die predictions / Wahrscheinlichkeiten für den neuen Token
new_token_logits = outputs.logits[:, -1, :]
new_token_logits.shape

In [ ]:
# mit .argmax() suchen wir den Token mit der höchsten Wahrscheinlichkeit -> das wird unser neuer Token
new_token = new_token_logits.argmax(dim=1)
tokenizer.decode(new_token)

### Text generieren mit Huggingface

Huggingface bietet uns dafür die .generate() Funktion - wir müssen das alles nicht manuell machen :)

In [ ]:
outputs = model.generate(**inputs, max_length=10)
text = tokenizer.batch_decode(outputs)[0]
text

### Text generieren ist eine "for loop"

wir können das aber auch manuell - Text generieren mit LLMs ist eine "for-loop".

In [ ]:
inputs = tokenizer('''Elvis Aaron Presley (* January 8, 1935 in''', return_tensors="pt", return_attention_mask=False)

for i in range(10):
    # Neuen Token generieren
    outputs = model(**inputs)
    new_token_logits = outputs.logits[:, -1, :]
    new_token = new_token_logits.argmax(dim=1)

    # Anhängen des neuen Tokens an die Inputs
    inputs["input_ids"] = torch.cat((inputs["input_ids"], new_token.unsqueeze(-1)), dim=1)

In [ ]:
tokenizer.decode(inputs["input_ids"][0])